In [ ]:
import tensorcircuit as tc
import numpy as np

# （待替换）
def modp_add(c, x_reg, y_reg, p, n):
    """
    模加 |x⟩|y⟩ → |x⟩|x+y mod p⟩
    参数:
        c: 量子电路
        输入寄存器（量子比特位置索引）:
        x_reg = [0, 1, ..., n-1]
        y_reg = [n, n+1, ..., 2n-1]
        其中 n 是每个寄存器的量子比特数, 电路c包括2n个量子比特
        p: 模数
    """
    return c

# （待替换）
def modp_add_constant(c, x_reg, constant, p, n):
    """
    模加 |x⟩→|x+constant mod p⟩
    参数:
        c: 量子电路
        输入寄存器（量子比特位置索引）:
        x_reg = [0, 1, ..., n-1]
        其中 n 是每个寄存器的量子比特数, 电路c包括n个量子比特
        p: 模数
    """
    return c

# （待替换）
def modp_double(c, x_reg, p, n):
    """
    模双倍 |x⟩ → |2x mod p⟩
    """
    return c

# （待替换）
def modp_minus(c, x_reg, p, n):
    """
    模负 |x⟩ → |-x mod p⟩
    """
    return c


def modp_mult(c, x_reg, y_reg, x_aux_reg, res_reg, p, n):
    """
    模乘法 |x⟩|y⟩|0⟩ → |x⟩|y⟩|x*y mod p⟩
    参数:
        c: 量子电路
        x_reg, y_reg: 输入寄存器（量子比特位置索引）
        x_reg = [0, 1, ..., n-1]
        y_reg = [n, n+1, ..., 2n-1]
        结果寄存器 res_reg = [2n, 2n+1, ..., 3n-1]
        其中 n 是每个寄存器的量子比特数, 电路c包括3n个量子比特
        例如 n=3 时，x_reg = [0,1,2], y_reg = [3,4,5], res_reg = [6,7,8]
        p: 模数
    """
    # 1. 通过连加计算 x*y=Σ (x*2^i) * y_i
    for i in range(n):
        # 对 y 的每一位 y_i，若为1，则|x⟩|y⟩|psi>->|x⟩|y⟩|psi+x*2^i mod p>
        # 使用c.control实现受控模加法门，注意为了保证可逆性不能忽略这一步
        with c.control(y_reg[i]):
            for j in range(i):
                modp_double(c, x_aux_reg, p, n)
            modp_add(c, x_aux_reg, res_reg, p, n)
    
    # 2. 结果已在 res_reg 中（x*y mod p）
    return c

def modp_square(c, x_reg, res_reg, p, n):
    """
    模平方 |x⟩|0⟩ → |x⟩|x² mod p⟩
    """
    for i in range(n):
        # 对 x 的每一位 x_i，若为1，则|x⟩|psi>->|x⟩|psi+x*2^i mod p>
        # 使用c.control实现受控模加法门，注意为了保证可逆性不能忽略这一步
        with c.control(x_reg[i]):
            for j in range(i):
                modp_double(c, x_reg, p, n)
            modp_add(c, x_reg, res_reg, p, n)
            modp_minus(c, x_reg, p, n)
        
    return c

